## Chapter 9: Number Theory

This chapter uses number theory to investigate more ideas about scientific computation.  We'll solve some problems related to prime numbers and other interesting this in number theory like perfect numbers and happy numbers. This chapter is used as an example of some of the ideas in scientific computing applied to number theory.

### 9.1: Prime Numbers

Recall that a prime number is an integer greater than 1 whose only factors are 1 and itself.

In [ ]:
findAllFactors(n::Integer) = filter(k -> n % k == 0, 1:n)

This function finds all of the factors of an integer. The result is an array of all of the factors: 

In [ ]:
findAllFactors(24)

In [ ]:
findAllFactors(50)

In [ ]:
findAllFactors(29)

Try to create a function `isPrime` that determines if its input is prime or not.  We will use the `findAllFactors` function that we wrote above. 

In [ ]:
function isPrime(n::Integer)
  local factors = findAllFactors(n)
  length(factors) == 2
end

In [ ]:
isPrime(13)

In [ ]:
isPrime(15)

In [ ]:
isPrime(1)

Write a more compact version of `isPrime`

In [ ]:
isPrime(n::Integer) = length(findAllFactors(n)) == 2

### 9.2: Finding Primes

It's also helpful to perhaps have a function that produce the next prime given any number:

In [ ]:
nextPrime(n::Integer) = isPrime(n+1) ? n+1 : nextPrime(n+1)

This is recursive.  It first's test if the next integer `n+1` is prime, if so, return `n+1`, else return `nextPrime(n+1)`

Let's test this a bit:

In [ ]:
nextPrime(10)

In [ ]:
nextPrime(47)

In [ ]:
nextPrime(1_000_000)

### HW #2 feedback

range compound boolean

In [ ]:
x = 12
if 1 <= x <= 15
    @show x
end

No need to put return type on functions


In [ ]:
function f(x::Real)::Real
    x^2
end

### 9.3 Perfect Numbers

A number is perfect if the sum of its factors (except itself) is itself.  For example, 6 is perfect because it factors are 1,2,3,6 and the sum of the first 3 numbers is 6. We seek to write a function `isPerfect` to test this.

In [ ]:
function isPerfect(n::Integer)
  # find the factors of n and check the sum
  factors = findAllFactors(n)
  sum(factors)-n == n
end

In [ ]:
isPerfect(6)

In [ ]:
isPerfect(28)

In [ ]:
findAllFactors(28)

In [ ]:
isPerfect(10)

In [ ]:
filter(isPerfect,1:100)

In [ ]:
isPerfect2(n::Integer) = sum(findAllFactors(n)) == 2n

In [ ]:
isPerfect2(28)

Let's time these two functions to see which is faster.  

In [ ]:
@time isPerfect(10_000)

In [ ]:
@time isPerfect2(10_000)

Notice that `@time` macro is reporting, we're going to use a package called `BenchmarkTools`.  We just did a tutorial for package managing in julia.  If you need a refresher see Appendix B of the textbook.

In [33]:
using BenchmarkTools

In [ ]:
@btime isPerfect(10_000)

In [ ]:
@btime isPerfect2(10_000)

The `@btime` macro runs the command a number of times and reports the average.  This helps in that since other things are happening on your computer, this is more accurate. Also, is using milliseconds (ms), microseconds (μs) and nanoseconds (ns). 

### Happy Numbers

Here’s another fun example. A number n is called happy if you perform the following steps:
1. Take the digits of n and square each one.
2. Sum the squares.
3. If the sum is 1, then the number is happy. If not repeat these steps.

* 13 is happy
* 19 is happy
* 4 is not happy

In [ ]:
function isHappy(n::Integer)
  if n==1
    return true
  elseif n==4
    return false
  else
    local d = digits(n)
    local sum = 0
    for i in d
      sum += i^2
    end
    return isHappy(sum)
  end
end

In [ ]:
digits(5398)

In [ ]:
isHappy(10)

In [ ]:
isHappy(11)

In [ ]:
isHappy(19)

In [ ]:
isHappy(58)

In [ ]:
function isHappy2(n::Integer)
  if n==1
    return true
  elseif n==4
    return false
  else
    return isHappy2(sum(x->x^2,digits(n)))
  end
end

In [ ]:
isHappy2(10)

In [ ]:
isHappy2(11)

In [ ]:
isHappy3(n::Integer) = n == 1 ? true : (n == 4 ? false : isHappy3(sum(x->x^2,digits(n))))

In [ ]:
isHappy3(10)

In [ ]:
isHappy3(11)

You will time this in your homework using the Benchmark tools

Let's find which of the first 100 whole numbers are happy.

In [ ]:
[isHappy(n) for n=1:100]

Although, you can figured it out a bit (the 1s mean the number is happy, 0 means it's not).  The following is a big easier:

In [ ]:
filter(isHappy, 1:100)

Remember what just happened here.  The numbers 1 to 100 (as a range) are filtered with only those that are true with the `isHappy` command.

### 8.5: Primes can be big!!

The following is a prime number:

In [ ]:
n = big(2)^89-1

which doesn't fit in `Int64`, which is why we used a `BigInt`.   Don't try to run `isPrime` on this number though. Trust me it won't finish because our algorithm isn't very good.  The largest known prime (as of mid 2020) is 24,862,048 digits long. 

### 8.6 speeding up our algorithms

If we benchmark time our original `isPrime` function on a reasonably sized prime number (but 1 million is tiny in prime number land), we get:

In [52]:
using BenchmarkTools

In [ ]:
@btime isPrime(1_000_003)

#### Speeding up `findAllFactors`

Since `isPrime` just uses `findAllFactors`, we should first concentrate on improving `findAllFactors`.  First, let's use a for loop instead.  It may be faster.

In [ ]:
function findAllFactors2(n::Integer)
  local factors = [1]
  for i=2:n-1
    if mod(n,i)==0
      push!(factors,i)
    end
  end
  push!(factors,n) # n is always a factor of itself
end

Let's time this versus the previous `findAllFactors`:

In [ ]:
@btime findAllFactors(10_000)

Note that we have supressed the output using the ; because we're more interested in the timing of the function rather than the result.

In [ ]:
@btime findAllFactors2(10_000);

Next, we really don't need to check all the way to n-1.  Let's stop earlier.

In [ ]:
function findAllFactors3(n::Integer)
    local factors = [1]
    for i=2:n/2
      if mod(n,i)==0
        push!(factors,i)
      end
    end
    push!(factors,n) # n is always a factor of itself
  end

And do some testing of the code.

In [ ]:
@btime findAllFactors3(10_000);

It's surprisingly slower for some values?  This is subtle, but because we used floating point division turns everything to floats, which are slower than ints. 


In [ ]:
function findAllFactors4(n::Integer)
    local factors = [1]
    for i=2:n ÷ 2
      if mod(n,i)==0
        push!(factors,i)
      end
    end
    push!(factors,n) # n is always a factor of itself
  end

In [ ]:
@btime findAllFactors4(10_000)

In [ ]:
findAllFactors4(100)

#### keep eliminating factors:

Factors nearly always come in pairs.  Consider all factors of 200:

```
1    2    4   5   8   10  
200  100  50  40  25  20
```


In [ ]:
function findAllFactors5(n::Integer)
  local x = round(Int,sqrt(n)) # closest integer to sqrt(n)
  local factors = [1,n]
  local j=2 # keep track where to insert elements
  for k=2:x
    if n%k==0
      # insert the new factors in the middle of the factors array
      splice!(factors,j:(j-1), k == x ? [k] : [k,div(n,k)])
      j+=1
    end
  end
  factors
end

In [ ]:
findAllFactors4(200)

In [ ]:
findAllFactors(100)

In [ ]:
@btime findAllFactors5(10_000);

This shows a speed up of about 12 for this example.  This doesn't scale linearly because we stop at $\sqrt{n}$

Let's now check the various `isPrime` functions

In [ ]:
function isPrime(n::Integer)
  return length(findAllFactors(n))==2
end

In [ ]:
function isPrime2(n::Integer)
  return length(findAllFactors3(n))==2
end

In [ ]:
function isPrime3(n::Integer)
  return length(findAllFactors4(n))==2
end

In [ ]:
function isPrime4(n::Integer)
    return length(findAllFactors5(n))==2
  end

In [ ]:
@btime isPrime(1_000_003)

In [ ]:
@btime isPrime2(1_000_003)

In [ ]:
@btime isPrime3(1_000_003)

In [ ]:
@btime isPrime4(1_000_003)

#### if we are determining primeness, why find all factors

Let's just walk through all integers up to $\sqrt{n}$ and stop if we find a factor

In [ ]:
function isPrime5(n::Integer)
  if n==1
    return false
  end
  local x = round(Int,sqrt(n)) # find integer nearest sqrt(n)
  for k=2:x
    if n%k==0
      return false
    end
  end
  true
end

Let's check this with a larger prime:

In [ ]:
@btime isPrime4(100_000_007)

In [ ]:
@btime isPrime5(100_000_007)

These look to be nearly the same.  Note that `isPrime3` uses an array but still is about as fast.

We can cut down a bit more, by only checking odd numbers. 

In [ ]:
function isPrime6(n::Integer)
  if n == 2
    return true
  elseif n%2==0
    return false
  end
  for k=3:2:round(Int,sqrt(n))
    if n%k==0
      return false
    end
  end
  true
end

In [ ]:
@btime isPrime6(100_000_007)

The following gets all primes up to `n` using the Sieve of Eratosthenes.  

In [ ]:
function getPrimes(n::Integer) ## return all primes up to n using the sieve of erathones
  local is_prime=trues(n) ## assume all are prime
  local k=2
  while true
    is_prime[2*k:k:n] .= false
    k = findnext(is_prime,k+1) # find the next prime after k
    if isnothing(k)
      return findall(is_prime)[2:end]
    end
  end
end

In [ ]:
getPrimes(25)

In [ ]:
function isPrime7(n::Integer)

end


In [ ]:
@btime isPrime7(100_000_007)

#### using the Primes package

In [91]:
using Primes

In [ ]:
@btime isprime(100_000_007)

In [ ]:
@btime isPrime6(100_000_007)

In [ ]:
@btime isprime(big(2)^89-1)

In [ ]:
filter(isPrime5,1_000_000_000_000:1_000_001_000_000)